In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, LabelEncoder, QuantileTransformer, Binarizer, KBinsDiscretizer
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score, classification_report, make_scorer, precision_score, recall_score
from sklearn.feature_selection import SelectFromModel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

%config InlineBackend.figure_format = 'retina'

# __Name__

### Zachary Dougherty

# __Research Question__

### Can we predict employee attrition (when employees leave the company)? 

# __Load the Data__

In [6]:
types = {
    'Age':'Int64',
    'BusinessTravel':'category',
    'DailyRate':'Int64',
    'Department':'category',
    'DistanceFromHome':'Int64',
    'Education':'category',
    'EducationField':'category',
    'EnvironmentSatisfaction':'category',
    'Gender':'category',
    'HourlyRate':'Int64',
    'JobInvolvement':'category',
    'JobLevel':'category',
    'JobRole':'category',
    'JobSatisfaction':'category',
    'MaritalStatus':'category',
    'MonthlyIncome':'Int64',
    'MonthlyRate':'Int64',
    'NumCompaniesWorked':'Int64',
    'OverTime':'category',
    'PercentSalaryHike':'Int64',
    'PerformanceRating':'category',
    'RelationshipSatisfaction':'category',
    'StockOptionLevel':'category',
    'TotalWorkingYears':'Int64',
    'TrainingTimesLastYear':'category',
    'WorkLifeBalance':'Int64',
    'YearsAtCompany':'Int64',
    'YearsInCurrentRole':'Int64',
    'YearsSinceLastPromotion':'Int64',
    'YearsWithCurrManager':'Int64',
    'Attrition':'category'
}

In [7]:
attrition = pd.read_csv('attrition_train.csv', dtype=types)
attrition.sample(5)

,Unnamed: 0,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
548,227,29,Travel_Frequently,1413,Sales,1,1,Medical,2,Female,42,3,3,Sales Executive,4,Married,7918,6599,1,No,14,3,4,1,11,5,3,11,10,4,1,No
585,201,49,Non-Travel,1002,Research & Development,18,4,Life Sciences,4,Male,92,3,2,Manufacturing Director,4,Divorced,6804,23793,1,Yes,15,3,1,2,7,0,3,7,7,1,7,No
111,900,36,Travel_Frequently,469,Research & Development,3,3,Technical Degree,3,Male,46,3,1,Research Scientist,2,Married,3692,9256,1,No,12,3,3,0,12,2,2,11,10,0,7,No
836,439,31,Travel_Frequently,534,Research & Development,20,3,Life Sciences,1,Male,66,3,3,Healthcare Representative,3,Married,9824,22908,3,No,12,3,1,0,12,2,3,1,0,0,0,Yes
1079,296,18,Travel_Rarely,230,Research & Development,3,3,Life Sciences,3,Male,54,3,1,Laboratory Technician,3,Single,1420,25233,1,No,13,3,3,0,0,2,3,0,0,0,0,Yes


In [8]:
# from pd report, we know that some columns are just constant or not relevant
y = attrition.Attrition.values
X = attrition.drop(['Attrition','Unnamed: 0'], axis=1)

In [9]:
# fix y, 1 = 'Yes', 0 = 'No'
y = LabelEncoder().fit_transform(y)

In [10]:
# class imbalance
print('Train',pd.Series(y).value_counts() / y.shape[0], sep='\n'); print()

Train
0    0.840136
1    0.159864
dtype: float64



# __Baseline__

We are going to train some models to get a baseline performance so we can evaluate the success or failuer of our feature engineering. Since we want to find all employees who may want to leave the company, we want to optimize for the True Negative Rate or specificity. Since sklearn doesn't have specifity as a default metric, we will create it here.

In [11]:
# defining the specificity metric
def specificity(y_true, y_pred):
    TN = np.sum(y_pred[y_true == 0] == 0) # find all correctly predicted neg values
    FP = np.sum(y_pred[y_true == 0] == 1) # find all falsely predicted pos values
    return TN / (TN + FP)

specificity_score = make_scorer(specificity)

In [12]:
cat_cols = np.where(X.dtypes == "category")[0]
num_cols = np.where(X.dtypes == "Int64")[0]

In [13]:
# Need to OHE for LogisticRegression 
tree_ct = ColumnTransformer([
    ('ord_enc', OrdinalEncoder(), cat_cols),
    ('sc', StandardScaler(), num_cols)
])

linear_ct = ColumnTransformer([
    ('ohe', OneHotEncoder(), cat_cols),
    ('sc', StandardScaler(), num_cols)
])

In [60]:
models = [
    RidgeClassifier,
    LogisticRegression,
    DecisionTreeClassifier,
    RandomForestClassifier,
    ExtraTreesClassifier,
    SGDClassifier,
    GradientBoostingClassifier,
    SVC
]

# class imbalance from observed values
params ={
    'class_weight': {0:0.15, 1:0.85}
}

print("Model\t   Specificity\tRecall\tPrecision")
for model in models:
    if model == LogisticRegression or model == SGDClassifier or model == RidgeClassifier:
        pipe = Pipeline([
            ('pre', linear_ct),
            ('model', model(**params))
        ])
    elif model == GradientBoostingClassifier:
        pipe = Pipeline([
            ('pre', linear_ct),
            ('model', model())
        ])
    else:
        pipe = Pipeline([
            ('pre', tree_ct),
            ('model', model(**params))
        ])
    
    print(str(model())[:15],
          f"{np.mean(cross_val_score(pipe, X, y, scoring=specificity_score, cv=5)):.3f}",
          f"{np.mean(cross_val_score(pipe, X, y, scoring=make_scorer(recall_score), cv=5)):.3f}",
          f"{np.mean(cross_val_score(pipe, X, y, scoring=make_scorer(precision_score), cv=5)):.3f}",
          sep="\t"
         )

Model	   Specificity	Recall	Precision
RidgeClassifier	0.768	0.771	0.392
LogisticRegress	0.776	0.750	0.394
DecisionTreeCla	0.857	0.298	0.311
RandomForestCla	0.995	0.064	0.672
ExtraTreesClass	0.991	0.106	0.638
SGDClassifier()	0.694	0.818	0.398
GradientBoostin	0.971	0.250	0.638
SVC()	0.693	0.739	0.314


Our goal is to optimize for high Recall, because it is better to check in on an employee who has no desire to leave rather than to forget about employees who are very likey to leave, so we would rather over predict attrition than underpredict. Ideally, we want to have a high Recall score and fairly high Spceficity. Performing well according to both of these metrics tells us that our model is fairly good at separating the two classed of employees, those will likely leave a company and those who are likely to stay. The best candidate models with these given goals are the linear models, namely LogisticRegression (higher Specificity) and Ridge Logistic Regression (higher Recall). We will continue our model development with these candidate models  and SGDClassifier in mind.

### __Candidate Models__
- RidgeClassifier
- LogisticRegression
- SGDClassifier

# __Model Development__

In order to apply our standard transformers, we need to find the indices of all the transformed columns from our custom transformers. We will transform the data with our custom transformer pipeline and then feed the newly created data into our main Pipeline to apply standard transformations, namely StandardScaler and Categorical encoding techniques, so we can train some models. We will also build a "col_df" which allows us to easily view our column names and data types.

In [42]:
# this class allows us to use Pipelines
# which don't have a final model
class DummyEstimator(BaseEstimator):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X
    
    def predict(self, X, y=None):
        return X
    
class Debug(BaseEstimator):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        print(X[0,:])
        return X

### __Feature Engineering for Linear Models__

__1) Normality:__ Linear models expect approximately normal columns, so we will transform some skewed numeric columns into normally distributed columns:
- __DailyRate__: Uniform
- __HourlyRate__: Very strange distribution, close to uniform
- __MonthlyIncome__: Skewed right
- __MonthlyRate__: Uniform
- __TotalWorkingYears__: Skewed right

In [43]:
to_norm_cols = np.where((X.columns == "MonthlyIncome") | (X.columns == "MonthlyRate") | (X.columns == "TotalWorkingYears") | (X.columns == "DailyRate") |
                        (X.columns == "HourlyRate"))[0]

to_norm_ct = ("norm_qt", QuantileTransformer(output_distribution="normal"), list(to_norm_cols))

__2) Discretization:__ Though we have many numeric columns, they are pseudo-categorical, as they only contain integer values and have small ranges. There a few ways to deal with this problem, one we will try is discretizing these numeric columns into sub-groups. For instance, in the DistanceFromHome column, we can observe a large drop off in the frequency of values after about 2 and 10 miles, with approximately uniform distributions before and after these threshold values, so we can create 3 new categories, "close to home", "moderate distance", "far from home". We can choose to either encode new categories, or simply break up the values into an arbitrary number of bins. This has the simultaneous effects of reducing complexity in the model, reducing feature space, and extracting some real world meaning from the variable distributions. For these purposes, we will use the sklearn provided __Binarizer__ and __KBinsDiscretizer__ classes. Using KBinsDiscretizer, we can use "quantile" as our method so that each category will have approximately the same number of observations, creatin approximately the desired categories for columns like "DistanceFromHome". Here are descriptions for the columns I adjusted:
- __DistanceFromHome__: As mentioned, we are splitting up this column into 3, equally distributed bins.
- __NumCompaniesWorked__: Split up into people who have remained relatively stable and people who have moved to many different companies.
- __PercentSalaryHike__: There is a large drop off in frequency after 15%, with a steady decline after that. We will use 3 quantile bins
- __YearsAtCompany__: Most people have been at a given company for 10 years or less, so let's try a Binarizer for this threshold.
- __YearsSinceLastPromotion__: The frequency drops off significantly after 2 years, so we will try a Binarizer to represent people who have been recently promoted and those who have worked a long time without one.
- __YearsInCurrentRole and YearsWithCurrManager__: These two columns are almost identical in distribution and scale, so we don't really need both of them. We will discretize them differently to try and avoid multicollinearity.

In [44]:
# getting our distance bins as mentioned
distance_bins = ("dist_bins", KBinsDiscretizer(n_bins=3, strategy="quantile", encode="onehot"), list(np.where(X.columns == "DistanceFromHome")[0]))

# NumCompanies
numcomp_binzer = ("numcomp_binzer", Binarizer(threshold=1), list(np.where(X.columns == "NumCompaniesWorked")[0]))

# PercentSalaryHike
pctsal_bins = ("pctsal_bins", KBinsDiscretizer(n_bins=3, strategy="quantile", encode="onehot"), list(np.where(X.columns == "PercentSalaryHike")[0]))

# YearsAtCompany
yrs_binzer = ("yrs_binzer", Binarizer(threshold=10), list(np.where(X.columns == "YearsAtCompany")[0]))

# YearsSinceLastPromotion
promo_binzer = ("promo_binzer", Binarizer(threshold=2), list(np.where(X.columns == "YearsSinceLastPromotion")[0]))

# YearsInCurrentRole
curr_role_bins = ("curr_role_bins", KBinsDiscretizer(n_bins=4, strategy="quantile", encode="onehot"), list(np.where(X.columns == "YearsInCurrentRole")[0]))

# YearsWithCurrManager
curr_man_bins = ("curr_man_bins", KBinsDiscretizer(n_bins=5, strategy="uniform", encode="onehot"), list(np.where(X.columns == "YearsWithCurrManager")[0]))

Putting it all together, we will construct a ColumnTransformer to alter our current set of columns. This will be used as a first step in our pipeline and the output will be fed to our "standard" column transformer.

In [45]:
norm_quant_bin_ct = ColumnTransformer([
    to_norm_ct, distance_bins, numcomp_binzer, pctsal_bins, yrs_binzer, promo_binzer, curr_role_bins, curr_man_bins
], remainder="passthrough")

We need to adjust the columns used for our standard transformers since we have changed the data types of some of our columns and the ColumnTransformer doesn't retain the order of the original columns. Since some columns are of type "category", we need to convert them to numeric type. This can be accomplished by either casting the type of the column or using an OridnalEncoder(), though the latter technique is a bit redundant.

In [46]:
new_ohe_cols = [24, 25, 27, 29, 32, 34, 35, 36]
ordinal_cols = [26, 28, 30, 31, 33, 37, 38, 39, 40]
new_sc_cols  = [23]

In [47]:
linear_ct = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), new_ohe_cols),
    ('ord_enc', OrdinalEncoder(), ordinal_cols),
    ('sc', StandardScaler(), new_sc_cols)
], remainder="passthrough")

In [49]:
params = {
    'class_weight': {1:0.85, 0:0.15}
}

print("Model\t\tSpecificity\tRecall\tPrecision")
for model in [LogisticRegression, RidgeClassifier, SGDClassifier]:
    if model == LogisticRegression:
        params.update({"solver":"liblinear"})
    pipe = Pipeline([
                ('feat_eng', norm_quant_bin_ct),
                ('pre', linear_ct),
                ('model', model(**params))
            ])
    if model == LogisticRegression:
        del params["solver"]
    
    pipe.fit(X, y)

    print(str(pipe['model'])[:18],
        f"{np.mean(cross_val_score(pipe, X, y, scoring=specificity_score, cv=5)):.3f}",
        f"{np.mean(cross_val_score(pipe, X, y, scoring=make_scorer(recall_score), cv=5)):.3f}",
        f"{np.mean(cross_val_score(pipe, X, y, scoring=make_scorer(precision_score), cv=5)):.3f}",
        sep="\t"
    )

Model		Specificity	Recall	Precision
LogisticRegression	0.762	0.717	0.369
RidgeClassifier(cl	0.755	0.717	0.359
SGDClassifier(clas	0.665	0.765	0.300


So, most of these changes seem to have decreased model performance. We will try again, this time we will force all numeric columns to be normally distributed, and not binarize or discretize columns.

### __Feature Engineering for Linear Models Take 2__

__1) Normality:__ Let's just convert all numeric columns into normal columns
- __DailyRate__: Uniform
- __DistancFromHome__: Skewed right
- __PercentSalaryHike__: Skewed right
- __HourlyRate__: Very strange distribution, close to uniform
- __MonthlyIncome__: Skewed right
- __MonthlyRate__: Uniform
- __TotalWorkingYears__: Skewed right
- __YearsAtCompany__: Skewed right
- __YearsInCurrentRole__: Strange distribution
- __YearsSinceLastPromotion__: Right skewed
- __YearsWithCurrManager__: Similar to YearsInCurrentRole

In [29]:
cols_df = pd.DataFrame({"cols":X.columns, "idx": np.arange(X.shape[1])}).set_index("cols")
to_norm_cols = cols_df.loc[["DailyRate","DistanceFromHome","PercentSalaryHike","HourlyRate",
                           "MonthlyIncome","MonthlyRate","TotalWorkingYears","YearsAtCompany",
                           "YearsInCurrentRole","YearsSinceLastPromotion","YearsWithCurrManager"],
                           "idx"].values


to_norm_ct = ColumnTransformer([("norm_qt", QuantileTransformer(output_distribution="normal"), list(to_norm_cols))], remainder="passthrough")

__2) Categorical Encoding:__ Now we need to encode the rest of the columns, either One Hot Encoding for columns without a proper ordering, such as JobRole, or Ordinal Encoding for columns such as EducationLevel.

In [30]:
new_ohe_cols = [12, 13, 15, 17, 20, 22, 24, 25, 26, 27]
ordinal_cols = [14, 16, 18, 19, 21, 23, 28, 29]
new_sc_cols  = [11]

In [31]:
linear_ct = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), new_ohe_cols),
    ('ord_enc', OrdinalEncoder(), ordinal_cols),
    ('sc', StandardScaler(), new_sc_cols)
], remainder="passthrough")

In [32]:
params = {
    'class_weight': {1:0.85, 0:0.15}
}

print("Model\t\tSpecificity\tRecall\tPrecision")
for model in [LogisticRegression, RidgeClassifier, SGDClassifier]:
    if model == LogisticRegression:
        params.update({"solver":"liblinear"})
    pipe = Pipeline([
                ('feat_eng', to_norm_ct),
                ('pre', linear_ct),
                ('model', model(**params))
            ])
    if model == LogisticRegression:
        del params["solver"]
    
    pipe.fit(X, y)

    print(str(pipe['model'])[:18],
        f"{np.mean(cross_val_score(pipe, X, y, scoring=specificity_score, cv=5)):.3f}",
        f"{np.mean(cross_val_score(pipe, X, y, scoring=make_scorer(recall_score), cv=5)):.3f}",
        f"{np.mean(cross_val_score(pipe, X, y, scoring=make_scorer(precision_score), cv=5)):.3f}",
        sep="\t"
    )

Model		Specificity	Recall	Precision
LogisticRegression	0.761	0.734	0.370
RidgeClassifier(cl	0.752	0.745	0.368
SGDClassifier(clas	0.714	0.687	0.339


### __Feature Engineering Conclusion__
After many different attempts at feature engineering, I was not able to get better results from altering the distribution of columns or discretizing pseudo-numeric columns. This may be because in the process of discretization, we lose information about the columns. It seems that for this dataset, reducing the feature space did not improve performance, so for our hyperparameter tuning, we will carry on with a LogisticRegression model trained on one hot encoded categorical columns and z-normalized numeric columns.

# __Hyperparameter Tuning__

In [51]:
cat_cols = np.where(X.dtypes == "category")[0]
num_cols = np.where(X.dtypes == "Int64")[0]

In [101]:
linear_ct = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ('sc', StandardScaler(), num_cols)
])

In [102]:
model = Pipeline([
    ('pre', linear_ct),
    ('clf', RidgeClassifier(class_weight={0:0.2, 1:0.8}))
])

model.fit(X, y)
np.mean(cross_val_score(model, X, y, scoring=make_scorer(recall_score), cv=5))

0.7284495021337127

In [170]:
model = Pipeline([
    ('pre', linear_ct),
    ('clf', RidgeClassifier())
])


params = {
    "clf__alpha": [0.001,0.01,0.01,1,10,100],
    "clf__class_weight": [{1:wgt, 0:1-wgt} for wgt in np.linspace(0.7, 0.9, 20)],
    "clf__solver": ["auto","svd","lsqr","sage"]
}

cv = RandomizedSearchCV(estimator=model,
                        param_distributions=params,
                        n_iter=30,
                        cv=5,
                        scoring=make_scorer(recall_score),
                        n_jobs=-1,
                        )

cv.fit(X, y)
cv.best_score_

0.8139402560455192

In [171]:
cv.best_estimator_

Pipeline(steps=[('pre',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  array([ 1,  3,  5,  6,  7,  8, 10, 11, 12, 13, 14, 18, 20, 21, 22, 24])),
                                                 ('sc', StandardScaler(),
                                                  array([ 0,  2,  4,  9, 15, 16, 17, 19, 23, 25, 26, 27, 28, 29]))])),
                ('clf',
                 RidgeClassifier(alpha=10,
                                 class_weight={0: 0.1210526315789473,
                                               1: 0.8789473684210527},
                                 solver='lsqr'))])

# __Model Evaluation__

In [172]:
# prepare test data
test = pd.read_csv('attrition_test.csv', dtype=types)

In [173]:
# from pd report, we know that some columns are just constant or not relevant
y_test = test.Attrition.values
X_test = test.drop(['Attrition','Unnamed: 0'], axis=1)

In [174]:
# fix y, 1 = 'Yes', 0 = 'No'
y_test = LabelEncoder().fit_transform(y_test)

Creating this dataframe allows us to easily select different columns of the same type for later transformations

In [175]:
final_model = cv.best_estimator_
final_model.fit(X, y)
pred = final_model.predict(X)
pred = final_model.predict(X_test)
recall_score(y_test, pred)

0.8163265306122449

In [176]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.95      0.69      0.80       245
           1       0.34      0.82      0.48        49

    accuracy                           0.71       294
   macro avg       0.65      0.75      0.64       294
weighted avg       0.85      0.71      0.75       294

